Setting the environment for MD calculation
Firstly, we need to install all necessary libraries and packages for our simulation. The main packages we will be installing are:

Anaconda (https://docs.conda.io/en/latest/miniconda.html)
OpenMM (https://openmm.org/)
PyTraj (https://amber-md.github.io/pytraj/latest/index.html)
py3Dmol (https://pypi.org/project/py3Dmol/)
Numpy (https://numpy.org/)
Matplotlib (https://matplotlib.org/)

# Only Execute the Next Three Cells if Using Google Colab

In [1]:
#@title **Install Conda Colab**
#@markdown It will restart the kernel (session), don't worry. Only execute this if running experiment in Google Colab, not if you are executing the code on your local machine!
!pip install -q condacolab
import condacolab
condacolab.install()

RuntimeError: This module must ONLY run as part of a Colab notebook!

In [ ]:
#@title ### **Import Google Drive**
#@markdown Click in the "Run" buttom to make your Google Drive accessible. Only run this command if in Google Colab!
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title **Check if you correctly allocated GPU nodes**
#@markdown Again, only run this command if you are running this code in Google Colab!

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

- 

In [3]:
#@title **Install dependencies**
#@markdown It will take a few minutes, please, drink a coffee and wait. ;-)
# install dependencies
#%%capture
!pip -q install py3Dmol
!pip install git+https://github.com/pablo-arantes/biopandas
!conda install mamba -c conda-forge -y
!mamba install -c conda-forge gcc=12.1.0 -y
!mamba install openmm -c conda-forge -y
!pip install --upgrade MDAnalysis
!pip install seaborn

#load dependencies
import sys
from biopandas.pdb import PandasPdb
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import urllib.request  
import numpy as np
import MDAnalysis as mda
import py3Dmol
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from matplotlib import colors
from IPython.display import set_matplotlib_formats

  Cloning https://github.com/pablo-arantes/biopandas to /tmp/pip-req-build-n51wr4_7
  Running command git clone --filter=blob:none --quiet https://github.com/pablo-arantes/biopandas /tmp/pip-req-build-n51wr4_7
  Resolved https://github.com/pablo-arantes/biopandas to commit 107e1a12491478242d373732a6fb3416b2569266
  Preparing metadata (setup.py) ... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0



# All requested packages already installed.


Looking for: ['gcc=12.1.0']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
pkgs/main/linux-64 ━━━━━━━━━━━━━━╸━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
pkgs/main/noarch  

In [ ]:
#@title **Set up the Systems**
#@markdown Create a box with water molecules with periodic boundary conditions
